In [78]:
# import the required libraries
import pandas as pd
import numpy as np
import urllib.request
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import requests
import json
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -q -c conda-forge folium=0.5.0 --yes
import folium

# assigning the required link to variable url and requesting html data
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page=urllib.request.urlopen(url)
soup=BeautifulSoup(page,"lxml")

# using beautifulSoup to find the coorect table
right_table=soup.find('table', class_='wikitable sortable')



# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [53]:
# creating empty lists to fill in the data from table
A=[]
B=[]
C=[]

# for loop to iterate through each row and fill each cell of the row with table data
for row in right_table.findAll("tr"):
    cells=row.findAll("td")
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

# converting the lists to dataFrame
df=pd.DataFrame(A,columns=['Postcode'])
df['Borough']=B
df['Neighborhood']=C

# removing rows where Boroughs are 'Not Assigned'
df_e=df.drop(df[(df['Borough']=='Not assigned')].index)

# merging 'Neighbourhood' column for same 'Postcode' and seperating by commas
df_g=df_e.groupby(['Postcode','Borough'])['Neighborhood'].apply(', '.join).str.replace('(\\n)', '').reset_index()

# replacing 'Neighbourhood' cells having values 'Not Assigned' to 'Borough' values
df_g.loc[df_g.Neighborhood == 'Not assigned', 'Neighborhood'] = df_g['Borough']

In [54]:
# obtaining the csv file with geospatial coordinates
!wget -q -O can_geo.csv https://cocl.us/Geospatial_data;

In [55]:
# extracting csv data into dataFrame and changing 'Post Code' column name to 'Postcode' to perform merge with 'df_g' dataFrame
can_df = pd.read_csv("can_geo.csv")
can_df=can_df.rename(columns = {'Postal Code':'Postcode'})

# innerjoin between two dataFrames with unique column 'Postcode'
df3 = pd.merge(df_g, can_df)

In [56]:
# creating dataFrame with boroughs having 'Toronto'
tor_df=df3[df3['Borough'].str.contains('Toronto')]
tor_df.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [85]:
# create map of Toronto using latitude and longitude values
map_tor = folium.Map(location=[43.6532, -79.3832], zoom_start=12)

# creating markers for different neighbourhoods in Toronto
for lat, lng, borough, neighborhood in zip(tor_df['Latitude'], tor_df['Longitude'], tor_df['Postcode'], tor_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_tor)

map_tor

In [58]:
# foursquare details
CLIENT_ID = 'OWSU25Z1WC3I1ENDAU5TWMWYEATSGVK45FZEKQJOCMX5PAJL'
CLIENT_SECRET = 'WKGSCJIZ3HX2XUOB2QMMHF0XREK2W0EUYR2SKA0AGJ40GBBU'
VERSION = '20190312'

In [59]:
# retrieving top venues nearby all neighbourhoods in Toronto in 500m radius
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["groups"][0]["items"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [60]:
# saving nearby venues into dataFrame
tor_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Clarks Corners, Sullivan, Tam O'Shanter
Agincourt North, L'Amoreaux East, Milliken, Steeles East
L'Amoreaux West, Steeles West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens, Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The D

In [86]:
# shape and first few rows of 'tor_venues' dataFrame
print(tor_venues.shape)
tor_venues.head()

(1075, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge, Malvern",43.806686,-79.194353,Interprovincial Group,43.805630,-79.200378,Print Shop
2,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,RIGHT WAY TO GOLF,43.785177,-79.161108,Golf Course
3,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum


In [62]:
# counting the number of rows
tor_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",20,20,20,20,20,20
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,2,2,2,2,2
"Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown",13,13,13,13,13,13
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Downsview North, Wilson Heights",19,19,19,19,19,19
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,20,20,20,20,20,20


In [63]:
# listing the total number for unique categories
print('There are {} uniques categories.'.format(len(tor_venues['Venue Category'].unique())))

There are 217 uniques categories.


In [64]:
# one hot encoding
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tor_onehot['Neighborhood'] = tor_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]

tor_onehot.head()

,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
# shape of 'tor_onehot' dataFrame
tor_onehot.shape

(1075, 217)

In [66]:
# grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category
tor_grouped = tor_onehot.groupby('Neighborhood').mean().reset_index()
tor_grouped

,Neighborhood,Yoga Studio,Accessories Store,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,...,0.00,0.00,0.05,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.076923,0.00,0.000000,0.0,0.0,0.00,0.000000
4,"Alderwood, Long Branch",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
5,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.052632,0.00,0.000000,0.0,0.0,0.00,0.000000
6,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
7,"Bedford Park, Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.050000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
8,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000
9,"Birch Cliff, Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.00,0.00,0.000000,0.00,0.000000,0.0,0.0,0.00,0.000000


In [68]:
# diplaying the shape of dataFrame 'tor_grouped'
tor_grouped.shape

(99, 217)

In [69]:
# printing each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in tor_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = tor_grouped[tor_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                  venue  freq
0      Asian Restaurant  0.10
1            Steakhouse  0.10
2  Gym / Fitness Center  0.05
3      Greek Restaurant  0.05
4                 Hotel  0.05


----Agincourt----
            venue  freq
0          Lounge  0.25
1  Clothing Store  0.25
2  Breakfast Spot  0.25
3    Skating Rink  0.25
4          Museum  0.00


----Agincourt North, L'Amoreaux East, Milliken, Steeles East----
            venue  freq
0      Playground   0.5
1            Park   0.5
2          Museum   0.0
3          Market   0.0
4  Massage Studio   0.0


----Albion Gardens, Beaumond Heights, Humbergate, Jamestown, Mount Olive, Silverstone, South Steeles, Thistletown----
                 venue  freq
0          Pizza Place  0.15
1        Grocery Store  0.15
2          Coffee Shop  0.08
3  Japanese Restaurant  0.08
4             Pharmacy  0.08


----Alderwood, Long Branch----
         venue  freq
0  Pizza Place  0.25
1  Coffee Shop  0.12
2          Gym  0.12
3

            venue  freq
0  Baseball Field   1.0
1     Yoga Studio   0.0
2          Museum   0.0
3  Massage Studio   0.0
4  Medical Center   0.0


----Fairview, Henry Farm, Oriole----
               venue  freq
0     Clothing Store  0.10
1           Tea Room  0.05
2          Juice Bar  0.05
3             Bakery  0.05
4  Electronics Store  0.05


----First Canadian Place, Underground city----
                  venue  freq
0                  Café  0.20
1           Coffee Shop  0.15
2            Steakhouse  0.10
3  Gym / Fitness Center  0.05
4           Art Gallery  0.05


----Flemingdon Park, Don Mills South----
                   venue  freq
0             Beer Store  0.10
1                    Gym  0.10
2       Asian Restaurant  0.10
3              Bike Shop  0.05
4  General Entertainment  0.05


----Forest Hill North, Forest Hill West----
              venue  freq
0          Bus Line   0.2
1              Park   0.2
2             Trail   0.2
3  Sushi Restaurant   0.2
4     Jewelry Store  

                           venue  freq
0                           Café  0.15
1                 Sandwich Place  0.10
2  Vegetarian / Vegan Restaurant  0.05
3              Jewish Restaurant  0.05
4                      BBQ Joint  0.05


----The Beaches----
               venue  freq
0        Coffee Shop  0.25
1  Health Food Store  0.25
2                Pub  0.25
3        Music Venue  0.00
4     Massage Studio  0.00


----The Beaches West, India Bazaar----
                  venue  freq
0                  Park  0.11
1           Pizza Place  0.06
2    Italian Restaurant  0.06
3  Fast Food Restaurant  0.06
4     Fish & Chips Shop  0.06


----The Danforth West, Riverdale----
                     venue  freq
0         Greek Restaurant  0.25
1           Ice Cream Shop  0.10
2       Italian Restaurant  0.10
3              Yoga Studio  0.05
4  Fruit & Vegetable Store  0.05


----The Junction North, Runnymede----
               venue  freq
0           Bus Line  0.25
1      Grocery Store  0.25
2  

In [70]:
# converting the top venues data into a dataFrame
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [71]:
# creating a dataFrame with top ten venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['Neighborhood']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Asian Restaurant,Steakhouse,Pizza Place,Noodle House,Coffee Shop,Plaza,Concert Hall,Café,Seafood Restaurant,Speakeasy
1,Agincourt,Clothing Store,Lounge,Skating Rink,Breakfast Spot,Cocktail Bar,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Curling Ice,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Pharmacy,Fried Chicken Joint,Beer Store,Coffee Shop,Video Store,Discount Store,Sandwich Place,Japanese Restaurant
4,"Alderwood, Long Branch",Pizza Place,Pharmacy,Coffee Shop,Pub,Sandwich Place,Skating Rink,Gym,Furniture / Home Store,Convenience Store,Dim Sum Restaurant


#### Run k-means to cluster the neighborhood into 4 clusters

In [93]:
# set number of clusters
kclusters = 4

tor_grouped_clustering = tor_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tor_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 2, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tor_merged = tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tor_merged = tor_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [92]:
# create map
map_clusters = folium.Map(location=[43.6532, -79.3832], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merged['Latitude'], tor_merged['Longitude'], tor_merged['Neighborhood'], tor_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [95]:
# cluster 1
tor_merged.loc[tor_merged['Cluster Labels'] == 0, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,0,Coffee Shop,Health Food Store,Pub,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
41,East Toronto,0,Greek Restaurant,Ice Cream Shop,Italian Restaurant,Yoga Studio,Brewery,Fruit & Vegetable Store,Indian Restaurant,Juice Bar,Dessert Shop,Cosmetics Shop
43,East Toronto,0,Coffee Shop,Café,Bakery,Seafood Restaurant,Juice Bar,Sandwich Place,Stationery Store,Italian Restaurant,Fish Market,Middle Eastern Restaurant
46,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gym / Fitness Center,Ice Cream Shop,Fast Food Restaurant,Diner,Dessert Shop,Mexican Restaurant
47,Central Toronto,0,Dessert Shop,Sushi Restaurant,Italian Restaurant,Gym,Indian Restaurant,Diner,Coffee Shop,Park,Café,Restaurant
48,Central Toronto,0,Playground,Restaurant,Tennis Court,Coffee Shop,Curling Ice,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner
49,Central Toronto,0,Coffee Shop,Pub,Pizza Place,Sushi Restaurant,Supermarket,Bagel Shop,Convenience Store,Fried Chicken Joint,American Restaurant,Sports Bar
51,Downtown Toronto,0,Restaurant,Café,Gift Shop,Pub,Diner,Jewelry Store,Deli / Bodega,Japanese Restaurant,Italian Restaurant,Bakery
52,Downtown Toronto,0,Pizza Place,Park,Salon / Barbershop,Bubble Tea Shop,Burger Joint,Restaurant,Ramen Restaurant,General Entertainment,Mexican Restaurant,Bookstore
53,Downtown Toronto,0,Coffee Shop,Park,Breakfast Spot,Bakery,Chocolate Shop,Pub,Restaurant,Mexican Restaurant,Dessert Shop,Performing Arts Venue


In [81]:
# cluster 2
tor_merged.loc[tor_merged['Cluster Labels'] == 1, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
42,East Toronto,1,Park,Gym,Sushi Restaurant,Fish & Chips Shop,Ice Cream Shop,Fast Food Restaurant,Italian Restaurant,Liquor Store,Movie Theater,Pet Store
45,Central Toronto,1,Gym,Hotel,Park,Burger Joint,Sandwich Place,Breakfast Spot,Dog Run,Food & Drink Shop,Clothing Store,Fruit & Vegetable Store
76,West Toronto,1,Supermarket,Pharmacy,Discount Store,Bakery,Pizza Place,Music Venue,Pool,Middle Eastern Restaurant,Café,Liquor Store
87,East Toronto,1,Light Rail Station,Yoga Studio,Butcher,Smoke Shop,Skate Park,Brewery,Burrito Place,Restaurant,Recording Studio,Park


In [87]:
# cluster 3
tor_merged.loc[tor_merged['Cluster Labels'] == 2, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,2,Park,Swim School,Bus Line,Women's Store,Dance Studio,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store
50,Downtown Toronto,2,Park,Playground,Trail,Cuban Restaurant,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
64,Central Toronto,2,Bus Line,Park,Jewelry Store,Trail,Sushi Restaurant,Dance Studio,Eastern European Restaurant,Drugstore,Dog Run,Discount Store


In [96]:
# cluster 4
tor_merged.loc[tor_merged['Cluster Labels'] == 3, tor_merged.columns[[1] + list(range(5, tor_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,3,Garden,Women's Store,Curling Ice,Electronics Store,Eastern European Restaurant,Drugstore,Dog Run,Discount Store,Diner,Dim Sum Restaurant
